In [ ]:
from dataclasses import dataclass
from datetime import date
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd

In [62]:
def rename_and_reorder_columns(df):
    """Renames and reorders the columns of the given DataFrame."""
    
    # Define a dictionary to map original column names to new column names
    column_name_mapping = {
        "id": "id",
        "نماد": "sym",
        "قیمت معامله شده هر ورقه": "price",
        "تاریخ آخرین روز معاملاتی": "last_date",
        "تاریخ سررسید": "mat_date",
        "YTM": "fbytm",
        "بازده ساده": "fbret",
    }
    
    # Rename the columns in the DataFrame
    df = df.rename(column_name_mapping, axis=1)
    
    # Reorder the columns in the DataFrame based on the new column names
    df = df[list(column_name_mapping.values())]

    return df


In [63]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd

def get_bonds_list():
    """Fetches bond data from the website and returns a DataFrame with the data."""
    
    # Initialize an empty list of bonds and the base URL
    bonds = []
    BASE_URL = "https://www.ifb.ir/ytm.aspx"

    # Make an HTTP request to the base URL
    req = Request(BASE_URL, headers={"User-Agent": "Mozilla/5.0"})
    html_page = urlopen(req).read()

    # Parse the HTML content
    soup = BeautifulSoup(html_page, "html.parser")

    # Initialize an empty DataFrame to store bond data
    df = pd.DataFrame()

    # Define bond classes and find all table elements in the HTML
    bonds_classes = ['KhazanehGrid', 'mGrid']
    tables = soup.find_all("table")
    
    # Filter tables based on bond classes
    tables = [table for table in tables if set(table["class"]) & set(bonds_classes)]

    # Iterate through the filtered tables and extract bond data
    for table in tables:
        # Convert the HTML table to a DataFrame
        new_df = pd.read_html(str(table))[0]

        # Extract bond IDs from the table
        hrefs = [(x['href'], x["symid"]) for x in table.find_all("a")]
        ids = [x[-1] for x in hrefs]

        # Check if the number of IDs matches the number of rows in the DataFrame
        assert len(ids) == new_df.shape[0]
        
        # Add the IDs to the new DataFrame and concatenate it with the main DataFrame
        new_df["id"] = ids
        df = pd.concat([df, new_df])

    return rename_and_reorder_columns(df)


In [81]:
df = get_bonds_list()
df['fv'] = 1_000_000

In [82]:
from persiantools import jdatetime
def to_greg(x):
    y, m, d = x.split('-')
    return jdatetime.JalaliDate(year=int(y), month=int(m), day=int(d)).to_gregorian()

In [83]:
df.dropna(subset=['last_date'], inplace=True)
df['greg_last_date'] = df['last_date'].apply(to_greg)
df['greg_mat_date'] = df['mat_date'].apply(to_greg)

today = date.today()
df = df[ (df['greg_last_date'] <= today)
        & (df['greg_mat_date'] >= today)]

In [84]:
df

,id,sym,price,last_date,mat_date,fbytm,fbret,fv,greg_last_date,greg_mat_date
0,25651,اخزا001,706900,1402-02-25,1403-08-21,26/1%,27/72%,1000000,2023-05-15,2024-11-11
1,25655,اخزا002,676600,1402-02-27,1403-10-24,26/48%,28/74%,1000000,2023-05-17,2025-01-13
2,25659,اخزا003,762000,1402-02-27,1403-04-18,26/79%,27/27%,1000000,2023-05-17,2024-07-08
3,25639,اخزا004,751000,1402-02-25,1403-05-22,25/82%,26/6%,1000000,2023-05-15,2024-08-12
4,25663,اخزا005,732000,1402-02-25,1403-06-26,26/16%,27/27%,1000000,2023-05-15,2024-09-16
...,...,...,...,...,...,...,...,...,...,...
318,28947,کرمان461,1010000,1401-11-30,1404-06-02,18/24%,NaN,1000000,2023-02-19,2025-08-24
319,28951,کرمان462,1003000,1402-02-25,1404-06-02,18/67%,NaN,1000000,2023-05-15,2025-08-24
322,25635,کیش0211,1000000,1401-06-07,1402-11-10,19/27%,NaN,1000000,2022-08-29,2024-01-30
323,30192,کیش05,1000000,1402-01-16,1405-07-24,19/24%,NaN,1000000,2023-04-05,2026-10-16
